In [1]:
import pandas as pd
import csv
import re

### 1.  Подготовка данных

In [2]:
train = pd.read_csv('train.csv',sep = '\t', quoting=csv.QUOTE_NONE, encoding='utf-8')
test = pd.read_csv('test.csv',sep = '\t', quoting=csv.QUOTE_NONE, encoding='utf-8')
dev = pd.read_csv('dev.csv',sep = '\t', quoting=csv.QUOTE_NONE, encoding='utf-8')

In [3]:
print('train:')
print('Примеров класса 1: ', int(train['class'].sum()))
print('Примеров класса 0: ', int(len(train['class'])-train['class'].sum()), '\n')
print('test:')
print('Примеров класса 1: ', int(test['class'].sum()))
print('Примеров класса 0: ', int(len(test['class'])-test['class'].sum()), '\n')
print('dev:')
print('Примеров класса 1: ', int(dev['class'].sum()))
print('Примеров класса 0: ', int(len(dev['class'])-dev['class'].sum()), '\n')

train:
Примеров класса 1:  2425
Примеров класса 0:  4900 

test:
Примеров класса 1:  680
Примеров класса 0:  1365 

dev:
Примеров класса 1:  956
Примеров класса 0:  1981 



### 2. Анализ разметки

In [4]:
train = train[train['class'] != 0]
test = test[test['class'] != 0]
dev = dev[dev['class'] != 0]

In [5]:
strange_cases = pd.DataFrame(columns=['text', 'class', 'cV', 'cR1', 'cR2', 'V', 'R1', 'R2'])

In [6]:
pattern = re.compile(r'^(\d+:\d+)$')

for index, row in test.loc[:,'cV':].iterrows():
    for i in row:
        if not re.match(pattern, str(i)):
            strange_cases = strange_cases.append(test.loc[index])

for index, row in train.loc[:,'cV':].iterrows():
    for i in row:
        if not re.match(pattern, str(i)):
            strange_cases = strange_cases.append(train.loc[index])
            
for index, row in dev.loc[:,'cV':].iterrows():
    for i in row:
        if not re.match(pattern, str(i)):
            strange_cases = strange_cases.append(dev.loc[index])

In [31]:
strange_cases = strange_cases.drop_duplicates()
strange_cases.head()

,text,class,cV,cR1,cR2,V,R1,R2
23,"Друзья делают твой смех немного громче, твою у...",1,7:13,14:23,24:38,52:52 79:79,40:51 68:78,52:64 79:92
126,При этом один из этих выводов используется для...,1,30:42,9:29,43:59,68:68 92:92,61:67 82:91,68:78 92:175
175,"- Невеста притворяется, что она девственница, ...",1,10:22,2:9,24:44,54:54 109:109,46:51 83:106,54:81 109:132
206,"Его деликатность была объявлена лицемерием, ег...",1,22:31,0:16,32:42,73:73 107:107,44:70 91:104,73:89 107:121
230,В бумажнике я обнаружил упаковку с тремя презе...,1,14:23,0:11,24:55,78:78 114:114,57:75 102:111,78:98 114:135


In [8]:
multiple_pairs = strange_cases.dropna(how='any')
multiple_pairs.shape

(274, 8)

В большинстве случаев, отклоняющихся от нормы (274), в ячейках содержится по две и более пар оффсетов, например:

In [10]:
strange_cases['text'][368]

'Именно поэтому прогресс в данном случае ассоциировался прежде всего с индустриальной экспансией, богатство — с накоплением финансовых резервов, а хозяйственный потенциал — с массой основных производственных фондов и доступных естественных ресурсов.'

Предикат _ассоциировался_ пропущен дважды

Ещё пример:

In [11]:
multiple_pairs['text'][600]

'По сравнению с декабрем 2005 г. индекс Доу-Джонса возрос на 2,9%, Xetra DAX-30 — на 9,7%, CAC-40 — на 8,8%, FTSE-100 — на 7,1%, Nikkei 225 — на 4,2%, Hang Seng — на 4,4%.'

In [12]:
multiple_pairs['V'][600]

'81:81 99:99 119:119 141:141 162:162'

предикат _возрос_ опущен 5 раз

Посмотрим на другие случаи

In [13]:
other_cases = pd.DataFrame(columns=['text', 'class', 'cV', 'cR1', 'cR2', 'V', 'R1', 'R2'])
for index, row in strange_cases.iterrows():
    for i in row:
        if 'nan' in str(i):
            other_cases = other_cases.append(row)

In [14]:
other_cases = other_cases.drop_duplicates()
other_cases.shape

(85, 8)

2 предложения оказались неразмеченными:

In [15]:
for index, row in other_cases.iterrows():
    if str(row['class']) == 'nan':
        print(row)
        other_cases = other_cases.drop(index)

text     Он с трудом передвигается на ногах, как если б...
class                                                  NaN
cV                                                     NaN
cR1                                                    NaN
cR2                                                    NaN
V                                                      NaN
R1                                                     NaN
R2                                                     NaN
Name: 7324, dtype: object
text     в июне 2011 года по сравнению с декабр
class                                       NaN
cV                                          NaN
cR1                                         NaN
cR2                                         NaN
V                                           NaN
R1                                          NaN
R2                                          NaN
Name: 2936, dtype: object


В остальных случаях пропущены cR2 и, соответсвенно, R2:

## !!!!!!Здесь требуется мнение экспертов

In [17]:
other_cases['text'][1262]

'Девушка поднимает глаза, смотрит на Нэша, потом – на меня.'

В этом случае субъект действия _девушка_ находится в другой клаузе **ЭТО НОРМ?**

In [18]:
other_cases['text'][852]

'Переступив через порог, я обнаружил небольшую площадку, спиральную лестницу, а наверху — стойку с инструментами в каменной будке.'

##### Здесь вообще всё хорошо,ничё не понимаю

In [27]:
for index, row in other_cases.iterrows():
    if str(row['cR2']) == 'nan':
        other_cases = other_cases.drop(index)

In [28]:
other_cases.shape

(0, 8)

Других случаев нет

#### Вывод

In [25]:
analysis = pd.DataFrame([['Две и более пары оффсетов', '274', strange_cases['text'][543]],\
                        ['Отсутствует cR2', '83', strange_cases['text'][1853]],\
                        ['Предложение не размечено', '2', strange_cases['text'][2936]]],
                        columns = ['Описание кейса', 'Количество случаев', 'Пример'])

In [26]:
analysis

,Описание кейса,Количество случаев,Пример
0,Две и более пары оффсетов,274,"В садике я мечтала о школе, в школе — об инсти..."
1,Отсутствует cR2,83,"Гарри в замешательстве уставился на Дамблдора,..."
2,Предложение не размечено,2,в июне 2011 года по сравнению с декабр
